Chapter 2.  Proof Basics
================

In **Prove-It**, proving an **expression** to be a true statement under some set of assumptions creates a **known truth** object and a **proof** object.  **Prove-It**, in its core, is agnostic to intrinsic truth-aptness of **expressions** except when they are proven and become "known truths".  The `proveit.logic` package defines the Boolean concept which may be used for making statements of the form $A \in \mathbb{B}$ (where $\mathbb{B}$ represents the Boolean set of True or False), but this is outside **Prove-It**'s core.  As far as the core is concerned, an **expression** is either proven or not. If an expression is not proven, it may or may not be truth-apt (*i.e.*, it may or may not be able to be shown to be true in some context).

A **known truth** contains assumptions, as **expressions**, and the single **expression** that is known to be a true statement under these assumptions.  It also has an associated **proof**.  The associated **proof** may get updated over time with a stronger proof (fewer assumption requirements) or newer proof that is as strong and has fewer proof steps.  A **proof** may also be disabled (via its `disable` method) to force an alternate or avoid its use.  Conveniently, a **known truth** object accesses the methods of the underlying proven **expression** seamlessly.  Using Python tricks (specifically, overloading the `__getattr__` and `__dir__` methods), the **known truth** behaves in most ways as if it were the known true **expression** it is wrapping.  This is useful because **expression** objects typically have convenience methods that are specific to the kind of expression, but it is also nice to display the object as a **known truth** (discussed next).

A **known truth** with some set of assumptions ${\cal A}$ is displayed in the form ${\cal A} \vdash S$ (or simply $\vdash S$ if ${\cal A}$ is empty), where $S$ is the (conditionally) true statement.  This single turnstile $\vdash$ is **Prove-It**'s indication that the **expression** on the right is derivable from the assumptions on the left.

For example, let us produce a trivial proof of the form $X \vdash X$:

In [1]:
from proveit._common_ import fx
%begin proof_basics
# fx is f(x) which is simply an Operation with a Variable operator and a Variable operand,
# but that is irrelevant.  It was chosen arbitrarily.
fx.exprInfo()

,core type,sub-expressions,expression
0,Operation,operator: 1operand: 2,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAACkAAAAVBAMAAAAp9toTAAAAMFBMVEX///8yMjLMzMwQEBAiIiJm Zma6urqYmJiqqqp2dnZERETc3NyIiIhUVFTu7u4AAAAe+HC4AAAAAXRSTlMAQObYZgAAAPpJREFU GNNjYCAAcqB0GYR6e3YDkGS6ABVlVQBTK/sLgCTjA5gmARDB9+0FiDoDN6oSRHB8ALPt4KKPweY4 gEg2AbgoVwMDA/tl3wCQ3AQGBu45t3tiGBg4QXz+AyAFjEAbXzKUFJgAjQRp2w92CXMCA8MEhvMM TUDDDID8frBh/EDRBwy3QEwmkPVT4KIMDDZw0bVgUWagxQzcnxj4oCbYQJwDtK2a5wPDOaCrBEBe A4sCncPxncuA+wDEZWz/wKJsDgx8a+bNPA5yJNAKHqinrJF9XAsyEB4mYDCHgcHwCZQND0k+oObK aCiHGxbqvBuQY2gPlD7EwAAAkqk1TOFGkRwAAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" />"
1,Variable,,
2,Variable,,


In [2]:
fxTruth = fx.prove(assumptions=[fx])

fxTruth: {f(x)} |- f(x)

The `prove` method attempts to prove that the **expression** is a true statement under the given set of assumptions and returns the **known truth** object if it is successful.  Automation strategies may be employed in the process and will be discussed in the <a href="tutorial08_automation.ipynb">automation</a> chapter.  In this case, $f(x)$ is trivially true if we use $f(x)$ as an assumption.  Again, **Prove-It** is agnostic to whether or not $f(x)$ is truth-apt.  This simply says that if $f(x)$ is assumed to be a true statement, then $f(x)$ is a true statement.  If `prove` is not successful, it raises a `ProofFailure` exception.

In [3]:
from proveit import ProofFailure
try:
    fx.prove()
except ProofFailure as e:
    print("EXPECTED ERROR: ", e)

EXPECTED ERROR:  Unable to prove f(x): 'conclude' method not implemented for proof automation


To obtain the **expression** from the **known truth**, and be able to display it as the **expression** only, access the `expr` attribute:

In [4]:
fxTruth.expr

f(x)

Its assumptions are accessible from its `assumptions` attribute (as a tuple).

In [5]:
fxTruth.assumptions

(f(x),)

But note that we may use the `KnownTruth` object as if it were the `Expression` object for most practical purposes.  It aquires all of the `Expression` object's attributes, except for some that it has overridden (such as `__class__`, `latex`, and `string`).

In [6]:
print(dir(fxTruth))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_addProof', '_checkIfReadyForQED', '_checkRelabelMap', '_checkedTruth', '_class_path', '_clear_', '_config_latex_tool', '_coreInfo', '_discardProof', '_equiv_method_evaluate_', '_equiv_method_evaluated_', '_equiv_method_evaluation_', '_equiv_method_simplification_', '_equiv_method_simplified_', '_equiv_method_simplify_', '_exprProofs', '_extractCoreInfo', '_extractExprClass', '_extractInitArgs', '_extractMyInitArgs', '_extractReferencedObjIds', '_extractStyle', '_formatted', '_formattedOperation', '_generate_unique_rep', '_genericExpr', '_implicitOperator', '_iterSubParamVals', '_make', '_meaningData', '_meaning_id', '_qed', '_repr

In [7]:
print(dir(fxTruth.expr))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_checkRelabelMap', '_class_path', '_clear_', '_config_latex_tool', '_coreInfo', '_equiv_method_evaluate_', '_equiv_method_evaluated_', '_equiv_method_evaluation_', '_equiv_method_simplification_', '_equiv_method_simplified_', '_equiv_method_simplify_', '_extractCoreInfo', '_extractExprClass', '_extractInitArgs', '_extractMyInitArgs', '_extractReferencedObjIds', '_extractStyle', '_formatted', '_formattedOperation', '_generate_unique_rep', '_genericExpr', '_implicitOperator', '_iterSubParamVals', '_make', '_meaningData', '_meaning_id', '_repr_html_', '_requirements', '_restrictionChecked', '_setContext', '_styleData', '_style_id', '_subExpressions'

In [8]:
fxTruth.operands

(x)

In [9]:
print(fxTruth.__class__) # this is a giveaway that it is not an actual Expression

<class 'proveit._core_.known_truth.KnownTruth'>


In [10]:
print(fxTruth.expr.__class__) # here it is

<class 'proveit._core_.expression.operation.function.Function'>


### Proof objects

Call the `proof` method of the **KnownTruth** to access the associated **Proof** object that proves the known truth.  A **Proof** is a DAG (directed, acyclic graph) of derivation steps that connect the known truth to *axioms*, *theorems*, or *assumptions*.

In [11]:
fxTruth.proof()

,step type,requirements,statement
0,assumption,,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAD8AAAAVBAMAAAADRiu8AAAAMFBMVEX///9ERETc3NyqqqpUVFS6 urrMzMwyMjIQEBCIiIh2dnYiIiLu7u5mZmaYmJgAAABJJ2g3AAAAAXRSTlMAQObYZgAAAU1JREFU KM9jYGDgu8eAHbCtUgBRPLuQBVknQGi+AJCKZhCT8wNYhMfTCUSlwFReAxHXQQRXA1iAcfE/EFUD U8AOIlYgKTgfYwAybgJMAd8CNAX2YJLjAEwBTwOagttgMhzhXGFUBU+6fUHULSDWmqI8F2jTWpgC 3gKwhp9gsg5o9gK+fRwbGBjMgdz3ILEMkI8ZmL6BFbQCTQR6nDmBgSEC5JULQI+3gWWYIaHxFWgC A4sAiJkPIsQfAFUmgF2yAaYA6JUJMAXcIL+ygN3A6ABWAA5b/QSGAxAr3iP0QnQxyAFDSGE+AydQ wXNUb54HOxXkN/sHEgyaDEjehCiIhwRhNAPDwVmnVgKdxlCKqmA6JPg4FLAHNZ+DHFIUgQHXBJgC PqACjpoNUHF4dB+HpweenUBFa2FGq8AUgJIPWyvE2MVISY4JmiA4E6BpEgDDJE+YBEt5uQAAAABJ RU5ErkJggg== "" style=""display:inline;vertical-align:middle;"" /> ⊢ <img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAACkAAAAVBAMAAAAp9toTAAAAMFBMVEX///8yMjLMzMwQEBAiIiJm Zma6urqYmJiqqqp2dnZERETc3NyIiIhUVFTu7u4AAAAe+HC4AAAAAXRSTlMAQObYZgAAAPpJREFU GNNjYCAAcqB0GYR6e3YDkGS6ABVlVQBTK/sLgCTjA5gmARDB9+0FiDoDN6oSRHB8ALPt4KKPweY4 gEg2AbgoVwMDA/tl3wCQ3AQGBu45t3tiGBg4QXz+AyAFjEAbXzKUFJgAjQRp2w92CXMCA8MEhvMM TUDDDID8frBh/EDRBwy3QEwmkPVT4KIMDDZw0bVgUWagxQzcnxj4oCbYQJwDtK2a5wPDOaCrBEBe A4sCncPxncuA+wDEZWz/wKJsDgx8a+bNPA5yJNAKHqinrJF9XAsyEB4mYDCHgcHwCZQND0k+oObK aCiHGxbqvBuQY2gPlD7EwAAAkqk1TOFGkRwAAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" />"


The **KnownTruth** that is proven by a **Proof** object is accessible via the `provenTruth` attribute.

In [12]:
fxTruth.proof().provenTruth

{f(x)} |- f(x)

The required sub-proofs are accessible via `requiredProofs`.  In this example, this is an empty list.  $\{f(x)\} \vdash f(x)$ has no requirements.

In [13]:
fxTruth.proof().requiredProofs # no requirements for {f(x)} |- f(x)

[]

This is a trivial example.  In the next few tutorial chapters, we will be examining the different kinds of derivation steps for making proofs and we will see more interesting examples of **known truths** and **proofs**.  For now, we simply note that **proof** display is similar to the way that the **expression** DAG is displayed using `exprInfo`.  The "root" node is the known truth that is proven and is indexed as `0`.  In general, with exception to trivial examples like this one, it will have requirements supplied as a list of other indices (similarly to the **sub-expressions** in `exprInfo`).

### 2 Additional Examples

As another simple example, consider the claim that $y$ is True if we know that $x$ is True and $x \Rightarrow y$ (an application of modus ponens).

As part of the example, we import `Variable` so we can give an example of defining the variables $x$ and $y$ from scratch, and we import the `Implies` expression constructor from proveit.logic. Then we prove $y$ as a result of the assumptions $x$ and $x \Rightarrow y$:

In [14]:
from proveit import Variable
x = Variable('x')
y = Variable('y')
from proveit.logic import Implies
yTruthFromModusPonens = y.prove(assumptions={x, Implies(x, y)})

yTruthFromModusPonens: {x => y , x} |- y

We can then look at the underlying proof, which is indeed a straightforward application of modus ponens:

In [15]:
yTruthFromModusPonens.proof()

,step type,requirements,statement
0,modus ponens,"1, 2","<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAF8AAAAVBAMAAADSoS/MAAAAMFBMVEX///9ERETc3NyqqqpUVFS6 urrMzMwyMjIQEBCIiIh2dnYiIiLu7u5mZmaYmJgAAABJJ2g3AAAAAXRSTlMAQObYZgAAAUdJREFU OMtjYGDgu8dAHGBbpQCieHYxEAvYmkEk5weiNTBcBxFcDcRrWIFdAw+pGtgPkKiB1YGwBq0pynMn wMVXQp22dgGDPsQ2qDxYA28BUGoB3z6ODUDO7d0gsP8CWBUX6weG+RCtUPn3IE5GAFAKGLjMCQir zQNAZDLvBgZBiFaoPDvQpJQ2cLiwCKCEUwVIxwGOCQw/oeEGlRd/wMAQATaZYwKyhvQ1YMr+ANs3 qAhEnnsBkGApADH1ExgOwP2wWwYWsZywIITIg/3ABfQMn8J8Bk5E6JtA6VoGRgdwNMLk4cFq/0CC QROunqkAyrjCEP+A4fwGhDxcw8FZp1Y+gGtgVIAyDs68r8Bw4hNCHkdMsyGYq4E4iGDSgAGWDTyg DPAATQMfTg3cCxgXoGoA5weenbg0MM+8CspgSG5tBVN8i0nJ0wDGd2PUojYrNgAAAABJRU5ErkJg gg== "" style=""display:inline;vertical-align:middle;"" /> ⊢"
1,assumption,,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEkAAAAVBAMAAAD4Ed5jAAAAMFBMVEX///9ERETc3NyqqqpUVFS6 urrMzMwyMjIQEBCIiIh2dnYiIiLu7u5mZmaYmJgAAABJJ2g3AAAAAXRSTlMAQObYZgAAARxJREFU KM9jYGDgu8eAB7CtUgBRPLsY8AK2ZhDJ+QG/KobrIIKrgYCqFdhV8RCliv0AMapYHbCrYlrpZHkT Ib4SQp26xcD0B6qKtwDIZ3j9oBvEub0bBPZfAKtaEA8OgPcgdkYAkM8wn8EMyRLzAFBAGdxmYBQA OhOoJ6UNKHCAwRHVlxUBIK92MvBPAPLEHzAwRCSAxDtRVKWvAZHMfxjygZLcC4BsFqC7GJi+QUIJ 4q7dMgxQF89ggLqLawMDwyOgK+cgjDKBhtsFBhGkkGD+x9jANAGuiKkAQvM6sP5DUsUjuWbVPIRR jApQP0jO/YQ77NngLJBrcKmCAW+GEwYwVXw4Ve1ikIKnL56duFRZzgVHQSuYw7eYYLoHAD7QTr3l h9LtAAAAAElFTkSuQmCC "" style=""display:inline;vertical-align:middle;"" /> ⊢ <img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAADcAAAAQBAMAAABAXPr7AAAAMFBMVEX///+6uroyMjJERETMzMwQ EBAiIiKYmJju7u5mZmaqqqpUVFSIiIh2dnbc3NwAAAAWgogoAAAAAXRSTlMAQObYZgAAAMNJREFU GNNjYEAFDQx4gAY+SWlU7tvTr04egPN4IUyOmxcY3jcwcFxgT2dNAPI508DgJ0QN4weG80CKgecD lwPCIC5FEOnFmcBgDDSAgdsAxZqmQJCjWQ8w/AHxWA8gy7Hogn2zvoHlC4h+7wD2HdTOZIhPJzHw BDAwsD84z8CD8DrXBgitycAMZK0vMGV4gzD1EZSezSBfwMDQfLb3TgFC0g5KN5+Z/wAjhBCeuoYz FLkTOKJwSrJdYL6AU5LrzEQgCQAs9DJLrF+J5gAAAABJRU5ErkJggg== "" style=""display:inline;vertical-align:middle;"" />"
2,assumption,,⊢


We can increase the complexity slightly, looking at a proof of $z$ given $x$, $x \Rightarrow y$, and $y \Rightarrow z$. The proof uses repeated application of modus ponens:

In [16]:
from proveit import Variable, ExprTuple
from proveit.logic import Implies
x = Variable('x')
y = Variable('y')
z = Variable('z')
someAssumptions = ExprTuple(Implies(x, y), Implies(y, z), x)
claim = z
claim.prove(assumptions=someAssumptions).proof()

,step type,requirements,statement
0,modus ponens,"1, 2","<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAJ0AAAAVBAMAAACu6/FQAAAAMFBMVEX///9ERETc3NyqqqpUVFS6 urrMzMwyMjIQEBCIiIh2dnYiIiLu7u5mZmaYmJgAAABJJ2g3AAAAAXRSTlMAQObYZgAAAfBJREFU OMtjYGDgu8dAFcC2SgFE8exioBJgawaRnB+oZR7DdRDB1UA181ZgN4+HgLYDJJrHfgC/eTUkmsfq gN+8EILmMa10sryJEF8JoU7dYmD6g8YC65gAZUS0MkGYWlOU506AmsdbAFTP8PpBN4hzezcI7L8A VrUgHhb5MBYvWHr3X4hxTHJzL0ICfAHfPo4NDAzvQZyMAKB6hvkMZkgONw8AJSeD2wyMAgwoLAhg LoXGG48PxMFA25gTgAEPdEdKGzjGHFFjuCIApLyTgX8CRCOMBQXHWyG0CixBsEBsE38ADIQEEKsT xbz0NWBn/GHIT2BAZUEyQhUkBbBegIlwgG3jXgAkWApAQfENkuog4bdbhgEasjMYUFjQ8NsITVBW cBv0E0CpEhx+XMCAfAQM7TkI+02g6fACgwgk9UJY8OCDpifmBQwsYGfwKcxn4DyASC/M/xgbmBDh w1QAdZUD6z8Gnt8wFhyow0Ll7btShvNA59g/kGDQREp/PJJrVs1DqGdUgAaz5NxPDDz9ClAWHAhB 6Qt8PwwYTgAlDs46tfIB7vzBhki5QMtZFWAsGEhAUR1EML/BgDfDCQNg4MBZ2MEDNPP4cJq3i0GK AWIehEXQPHD5x7MTl0rLuaBsMgHOwgo4kcIJksr5FlOx/gAA6VmSxnbKAJIAAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" /> ⊢"
1,assumption,,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEkAAAAVBAMAAAD4Ed5jAAAAMFBMVEX///9ERETc3NyqqqpUVFS6 urrMzMwyMjIQEBCIiIh2dnYiIiLu7u5mZmaYmJgAAABJJ2g3AAAAAXRSTlMAQObYZgAAAQ5JREFU KM9jYGDgu8eAB7CtUgBRPLsY8AK2ZhDJ+QG/KobrIIKrgYCqFdhV8RCliv0AMapYHbCr4lm7gEEf YcJKKB3RyjQBqoq3AKiS9QPDfBDn9m4Q2H8BrIhJbu5FIPUexM4IYGBI5t3AIIhkiXkAxBc8PiBn AvWktAEZBzgmMPxE9mUFWBmDCpgUfwC0PAHIsD/A9g1JVfoaiDfANnMvABIsBeDw5WxAuGu3DESx FZgEu4trA5CoZWBE8r4JhGJewMCCEl5XGOIfwBUxFUDtffuuFEXVwZn3FeCqGKHMC3w/DNDCfjVS WsERjywbeJrxpwm+BpBnGRfgT188O4G+mXkVd1ptBVN8iwmmewDdCUvOfhSdtgAAAABJRU5ErkJg gg== "" style=""display:inline;vertical-align:middle;"" /> ⊢"
2,modus ponens,"3, 4","<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAF8AAAAVBAMAAADSoS/MAAAAMFBMVEX///9ERETc3NyqqqpUVFS6 urrMzMwyMjIQEBCIiIh2dnYiIiLu7u5mZmaYmJgAAABJJ2g3AAAAAXRSTlMAQObYZgAAAUdJREFU OMtjYGDgu8dAHGBbpQCieHYxEAvYmkEk5weiNTBcBxFcDcRrWIFdAw+pGtgPkKiB1YGwBq0pynMn wMVXQp22dgGDPsQ2qDxYA28BUGoB3z6ODUDO7d0gsP8CWBUX6weG+RCtUPn3IE5GAFAKGLjMCQir zQNAZDLvBgZBiFaoPDvQpJQ2cLiwCKCEUwVIxwGOCQw/oeEGlRd/wMAQATaZYwKyhvQ1YMr+ANs3 qAhEnnsBkGApADH1ExgOwP2wWwYWsZywIITIg/3ABfQMn8J8Bk5E6JtA6VoGRgdwNMLk4cFq/0CC QROunqkAyrjCEP+A4fwGhDxcw8FZp1Y+gGtgVIAyDs68r8Bw4hNCHkdMsyGYq4E4iGDSgAGWDTyg DPAATQMfTg3cCxgXoGoA5weenbg0MM+8CspgSG5tBVN8i0nJ0wDGd2PUojYrNgAAAABJRU5ErkJg gg== "" style=""display:inline;vertical-align:middle;"" /> ⊢"
3,assumption,,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEkAAAAVBAMAAAD4Ed5jAAAAMFBMVEX///9ERETc3NyqqqpUVFS6 urrMzMwyMjIQEBCIiIh2dnYiIiLu7u5mZmaYmJgAAABJJ2g3AAAAAXRSTlMAQObYZgAAARxJREFU KM9jYGDgu8eAB7CtUgBRPLsY8AK2ZhDJ+QG/KobrIIKrgYCqFdhV8RCliv0AMapYHbCrYlrpZHkT Ib4SQp26xcD0B6qKtwDIZ3j9oBvEub0bBPZfAKtaEA8OgPcgdkYAkM8wn8EMyRLzAFBAGdxmYBQA OhOoJ6UNKHCAwRHVlxUBIK92MvBPAPLEHzAwRCSAxDtRVKWvAZHMfxjygZLcC4BsFqC7GJi+QUIJ 4q7dMgxQF89ggLqLawMDwyOgK+cgjDKBhtsFBhGkkGD+x9jANAGuiKkAQvM6sP5DUsUjuWbVPIRR jApQP0jO/YQ77NngLJBrcKmCAW+GEwYwVXw4Ve1ikIKnL56duFRZzgVHQSuYw7eYYLoHAD7QTr3l h9LtAAAAAElFTkSuQmCC "" style=""display:inline;vertical-align:middle;"" /> ⊢ <img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAADcAAAAQBAMAAABAXPr7AAAAMFBMVEX///+6uroyMjJERETMzMwQ EBAiIiKYmJju7u5mZmaqqqpUVFSIiIh2dnbc3NwAAAAWgogoAAAAAXRSTlMAQObYZgAAAMNJREFU GNNjYEAFDQx4gAY+SWlU7tvTr04egPN4IUyOmxcY3jcwcFxgT2dNAPI508DgJ0QN4weG80CKgecD lwPCIC5FEOnFmcBgDDSAgdsAxZqmQJCjWQ8w/AHxWA8gy7Hogn2zvoHlC4h+7wD2HdTOZIhPJzHw BDAwsD84z8CD8DrXBgitycAMZK0vMGV4gzD1EZSezSBfwMDQfLb3TgFC0g5KN5+Z/wAjhBCeuoYz FLkTOKJwSrJdYL6AU5LrzEQgCQAs9DJLrF+J5gAAAABJRU5ErkJggg== "" style=""display:inline;vertical-align:middle;"" />"
4,assumption,,⊢


Proofs of more interesting claims will need to wait until later tutorial chapters. What might seem like simple applications of modus tollens, for example, actually involve derived theorems, `Forall` expressions, and `Forall` specializations (universal instantiations), which are covered in later tutorials.

### Default assumptions

If you do not supply a set of assumptions into the `prove` method (or any other **Prove-It** method that accepts an `assumptions` argument), it will use `proveit._core_.defaults.defaults.assumptions` (aliased as `proveit.defaults.assumptions`).  By default, this "default" is the empty set, but it can be changed for convenience.

For example, we can repeat our earlier proof of $f(x)$ by shifting the assumption into the default assumptions and calling `.prove()` without explicitly including the assumption:

In [17]:
from proveit import defaults
defaults.assumptions = [fx]
newFxTruth = fx.prove()
newFxTruth

{f(x)} |- f(x)

In [18]:
newFxTruth.proof()

,step type,requirements,statement
0,assumption,,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAD8AAAAVBAMAAAADRiu8AAAAMFBMVEX///9ERETc3NyqqqpUVFS6 urrMzMwyMjIQEBCIiIh2dnYiIiLu7u5mZmaYmJgAAABJJ2g3AAAAAXRSTlMAQObYZgAAAU1JREFU KM9jYGDgu8eAHbCtUgBRPLuQBVknQGi+AJCKZhCT8wNYhMfTCUSlwFReAxHXQQRXA1iAcfE/EFUD U8AOIlYgKTgfYwAybgJMAd8CNAX2YJLjAEwBTwOagttgMhzhXGFUBU+6fUHULSDWmqI8F2jTWpgC 3gKwhp9gsg5o9gK+fRwbGBjMgdz3ILEMkI8ZmL6BFbQCTQR6nDmBgSEC5JULQI+3gWWYIaHxFWgC A4sAiJkPIsQfAFUmgF2yAaYA6JUJMAXcIL+ygN3A6ABWAA5b/QSGAxAr3iP0QnQxyAFDSGE+AydQ wXNUb54HOxXkN/sHEgyaDEjehCiIhwRhNAPDwVmnVgKdxlCKqmA6JPg4FLAHNZ+DHFIUgQHXBJgC PqACjpoNUHF4dB+HpweenUBFa2FGq8AUgJIPWyvE2MVISY4JmiA4E6BpEgDDJE+YBEt5uQAAAABJ RU5ErkJggg== "" style=""display:inline;vertical-align:middle;"" /> ⊢ <img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAACkAAAAVBAMAAAAp9toTAAAAMFBMVEX///8yMjLMzMwQEBAiIiJm Zma6urqYmJiqqqp2dnZERETc3NyIiIhUVFTu7u4AAAAe+HC4AAAAAXRSTlMAQObYZgAAAPpJREFU GNNjYCAAcqB0GYR6e3YDkGS6ABVlVQBTK/sLgCTjA5gmARDB9+0FiDoDN6oSRHB8ALPt4KKPweY4 gEg2AbgoVwMDA/tl3wCQ3AQGBu45t3tiGBg4QXz+AyAFjEAbXzKUFJgAjQRp2w92CXMCA8MEhvMM TUDDDID8frBh/EDRBwy3QEwmkPVT4KIMDDZw0bVgUWagxQzcnxj4oCbYQJwDtK2a5wPDOaCrBEBe A4sCncPxncuA+wDEZWz/wKJsDgx8a+bNPA5yJNAKHqinrJF9XAsyEB4mYDCHgcHwCZQND0k+oObK aCiHGxbqvBuQY2gPlD7EwAAAkqk1TOFGkRwAAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" />"


In [19]:
%end proof_basics

# Next chapter: <a href="tutorial03_implies.ipynb">Implication</a>

## <a href="tutorial00_introduction.ipynb#contents">Table of Contents</a>